# Guided dFBA with uptake constraints from NMR

## Import packages

In [28]:
import csv
import math

import cobra as cb
from cobra.flux_analysis import flux_variability_analysis as fva
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors
import openpyxl as xl

## Conversion factors

Conversion factors to transform signal to concentration.

These were determined by as described in the Methods section of our C.diff HRMAS pre-print. _These are estimates of concentration and should not be taken as precise values._

In [29]:
# Scale Factors - calculated from HMDB reference spectra
# = concentration / AVG(pk_intensities)
scale_f2 = {
    'glc' : 0.110683136*10,
    'ac'  : 0.089510168*10,
    'alaL': 0.089510168*10,
    'eto' : 0.110683136*10,
    'lacS': 0.089510168*10,
    'proL': 0.227765585,
    '5apn': 0.227765585,
}
scale_f = {
    'glc' : 1.,
    'ac'  : 1.,
    'alaL': 1.,
    'eto' : 1.,
    'lacS': 1.,
    'but' : 1.,
    'buta': 1.,
    'proL': 1.,
    '5apn': 1., #0.5
    'leuL': 1.,
    'ival': 1.,
    'isocap': 0.95,
}

## Some key functions

Helper functions:
1. Evaluate logistic function at timepoint.
1. Evaluate derivative of logistic function at timepoint.
1. Update the uptake/secretion reaction bounds with the logistic derivative. This function evaluates both the logistic curve and its derivative at the timepoint, updates the bounds, and returns both solutions. Pass in the logistic coefficients calculated with MATLAB.


In [30]:
def logistic (L, k, x0, C, x):
    """Evaluate the logistic function at point x."""
    return L/(1. + math.exp(-1*k*(x - x0))) + C

def ddx_logistic(L, k, x0, x):
    """Evaluate the derivative of the logistic function at point x."""
    return k*L*math.exp(-k*(x - x0))/(1. + math.exp(-1*k*(x - x0)))**2

def find_minmax_bounds(met, Ls, ks, x0s, Cs, t):
    all_bounds = []
    all_signal = []
    for L in Ls:
        for k in ks:
            for x0 in x0s:
                all_bounds.append(ddx_logistic(scale_f[met]*L, k, x0, t))
                for C in Cs:
                    all_signal.append(logistic(scale_f[met]*L, k, x0, scale_f[met]*C, t))
    return min(all_bounds), max(all_bounds), min(all_signal), max(all_signal)

def update_uptake_bounds(model, t, met, Ls, ks, x0s, Cs):
    """Update the uptake rate of exchange reaction at timepoint t."""
    logistic_sol = logistic(scale_f[met]*Ls[0], ks[0], x0s[0], scale_f[met]*Cs[0], t)
    uptake_rate = ddx_logistic(scale_f[met]*Ls[0], ks[0], x0s[0], t)
    uptake_lb, uptake_ub, signal_lb, signal_ub = find_minmax_bounds(met, Ls[1:], ks[1:], x0s[1:], Cs[1:], t)
    if met in ['glc', 'proL', 'leuL']:
        rid = 'Ex_' + met
        uptake_rate *= -1
        uptake_lb *= -1
        uptake_ub *= -1
        model.reactions.get_by_id(rid).upper_bound = uptake_ub
    elif met in ['ival', 'isocap', '5av']:
        rid = 'Sec_' + met
        model.reactions.get_by_id(rid).upper_bound = uptake_ub
    else:
        rid = 'Sec_' + met
    model.reactions.get_by_id(rid).lower_bound = uptake_lb
    return logistic_sol, uptake_rate, uptake_lb, uptake_ub, signal_lb, signal_ub

def update_dependent_uptakes(model, sol, resolution):
    """Update maximum uptake rates of unconstrained metabolites."""
    for exch in model.exchanges:
        if sol.fluxes[exch.id] > 0:
            exch.upper_bound = max(0, exch.upper_bound - sol.fluxes[exch.id]*sol.objective_value*resolution*0.5)

## Plot function

In [31]:
def areaplot(df, dl, du, t_max=48, ylabel='flux (mol/gDW/h)'):
    f = plt.figure(figsize=(14, 10), )#fontsize=30,
    ax = plt.axes(
        xticks=(range(0, t_max+1, 12)),
        xlim=(0, t_max),
        ylim=(0, 1.5),
    )
    for met, ser in df.iteritems():
        ser.index.name = 'index'
        p = ax.plot(ser.index.to_numpy(), ser.to_numpy(), '-', label=met, lw=5)
        color = matplotlib.colors.to_rgb(p[0].get_color())
        color = color + (0.2,)
        ax.fill_between(dl.index.to_numpy(), dl[met].to_numpy(), du[met].to_numpy(), color=color)
    ax.set_xlabel('time (h)', fontsize=30, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=30, fontweight='bold')
    plt.xticks(ax.get_xticks(), weight='bold')
    plt.yticks(ax.get_yticks(), weight='bold')
    
def niceplot(df, t_max=48, ylabel='flux (mol/gDW/h)'):
    ax = df.plot(
        figsize=(14, 10),
        xticks=(range(0, t_max+1, 12)),
        xlim=(0, t_max),
        ylim=(0, None),
        fontsize=30,
        lw=5,
    )
    ax.set_xlabel('time (h)', fontsize=30, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=30, fontweight='bold')
    plt.xticks(ax.get_xticks(), weight='bold')
    plt.yticks(ax.get_yticks(), weight='bold')
    


## Main dFBA program

In [32]:
def dfba_main(specsheet, traced, modelfile='../../data/icdf834-mhmc.json', t_max=48, resolution=1):
    """Main dFBA function. Computes successive FBA solutions and plots the concentration, uptake rates, and tracked reaction fluxes.
       
       Arguments:
       specsheet -- path to tsv sheet containing logistic parameters
       traced -- list of reaction ids to be traced over the timecourse
       
       Keyword arguments:
       modelfile -- location of metabolic model
       t_max -- end timepoint in hours (default 48)
       resolution -- number of hours per dFBA sample (default 1)
       """
    print(f"""dFBA log: Begin dFBA analysis with sheet
          {specsheet.split('/')[-1]}, endpoint
          {t_max} hours, and resolution {resolution}.""")
    
    # Load metabolic model and logistic fit specs #
    print('dFBA log: loading model and specsheet...')
    model = cb.io.load_json_model(modelfile)
    model.reactions.ID_90.lower_bound = -1000
    init_cnc = dict()
    for rxn in model.reactions:
        if rxn.id.startswith('Ex_') and rxn.id.endswith('L') or rxn.id in ['Ex_gly', 'Ex_his']:
            init_cnc[rxn.id] = rxn.upper_bound
            rxn.upper_bound *= 0.03
    #model.reactions.ID_53.upper_bound = 0
    #model.reactions.ID_326.upper_bound = 0
    model.reactions.ID_393.upper_bound = 0
    model.reactions.ID_393.lower_bound = 0
    params = pd.read_excel(specsheet, sheet_name='cf', header=0, index_col=0)
    par_lb = pd.read_excel(specsheet, sheet_name='lb', header=0, index_col=0)
    par_ub = pd.read_excel(specsheet, sheet_name='ub', header=0, index_col=0)
    
    # Initialize results dataframes
    timecourse = list(range(0, t_max + resolution, resolution))
    upta_lb = pd.DataFrame(np.zeros(shape=(len(timecourse), par_lb.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_lb.index])
    uptakes = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    upta_ub = pd.DataFrame(np.zeros(shape=(len(timecourse), par_ub.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_ub.index])
    sign_lb = pd.DataFrame(np.zeros(shape=(len(timecourse), par_lb.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_lb.index])
    sign_ub = pd.DataFrame(np.zeros(shape=(len(timecourse), par_ub.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_ub.index])
    signals = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    rxnflux = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced]) # 
    rxnflux2 = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced])
    
    print('dFBA log: simulation output begin.')
    mets = params.index.to_numpy()
    
    fulldata = [params.to_numpy(), par_lb.to_numpy(), par_ub.to_numpy()]
    for i, t in enumerate(timecourse):
        for j, met in enumerate(mets):
            data = [[a[j, b] for a in fulldata] for b in range(4)]
            logi, ddxlogi, ulb, uub, slb, sub = update_uptake_bounds(model, t, mets[j], *data)
            uptakes.at[t, model.metabolites.get_by_id(met + '_c').name] = ddxlogi
            upta_lb.at[t, model.metabolites.get_by_id(met + '_c').name] = ulb
            upta_ub.at[t, model.metabolites.get_by_id(met + '_c').name] = uub
            signals.at[t, model.metabolites.get_by_id(met + '_c').name] = logi
            sign_lb.at[t, model.metabolites.get_by_id(met + '_c').name] = slb
            sign_ub.at[t, model.metabolites.get_by_id(met + '_c').name] = sub
        #sol = cb.flux_analysis.pfba(model)
        #sol = cb.flux_analysis.geometric_fba(model)
        #sol = cb.flux_analysis.flux_variability_analysis(model)
        sol = model.optimize()
        if i % 10 == 0:  
            print(f'dFBA log: Time = {t}  (cycle {i}) \tFBA solution: {sol.fluxes["Ex_biomass"]}')
        if sol.fluxes['Ex_biomass'] < 0.0001:
            print(f'dFBA log: infeasible solution on cycle {i}.')
            for rid in traced:
                rxnflux.at[t, model.reactions.get_by_id(rid).name] = 0
        else:
            for rid in traced:
                rxnflux.at[t, model.reactions.get_by_id(rid).name] = abs(sol.fluxes[rid])
        #update_dependent_uptakes(model, sol, resolution)
        if i in [0, 6, 12, 24, 36, 48]:
            # print(model.summary(solution=sol))
            print("time = " + str(i))
#             print(model.summary(solution=sol))
#             print(model.metabolites.get_by_id('gluL_c').summary(solution=sol))
#             print(model.metabolites.get_by_id('serL_c').summary(solution=sol))
#             print(model.metabolites.get_by_id('hpyr_c').summary(solution=sol))
        #         for rxn in model.reactions:
#             if (rxn.id.startswith('Ex_') and rxn.id.endswith('L') or rxn.id in ['Ex_gly', 'Ex_his']):
#                 cnc = init_cnc[rxn.id]
#                 if cnc > 0:
#                     deplete = sol.fluxes[rxn.id] / cnc
#                     rxn.upper_bound *= (1 - deplete)
#                     cnc *= (1 - deplete)
        
            
    print(f'Complete after {i} cycles ({t} hours). Final flux: {sol.fluxes["Ex_biomass"]}')
    areaplot(uptakes, upta_lb, upta_ub, ylabel='flux (mM/h)')
    plt.show()
    niceplot(uptakes, ylabel='flux (mM/h)')
    plt.show()
#     plt.savefig('../../data/uptakes.png')
    niceplot(signals, ylabel='concentration (mM)')
    plt.show()
#     plt.savefig('../../data/concentrations.png')
    niceplot(rxnflux)
    plt.show()
#     plt.savefig('../../data/fluxes.png')
    rxnflux.to_csv('../../data/sumfluxes.csv', sep='\t')
#     rxnflux.to_excel('../../data/some_flux.xlsx', engine='xlsxwriter')
    writer = pd.ExcelWriter('../../data/uptakes.xlsx', engine='openpyxl')
    uptakes.to_excel(writer, sheet_name='data', engine='openpyxl')
    upta_lb.to_excel(writer, sheet_name='lbdev', engine='openpyxl')
    upta_ub.to_excel(writer, sheet_name='ubdev', engine='openpyxl')
    signals.to_excel(writer, sheet_name='signal', engine='openpyxl')
    sign_lb.to_excel(writer, sheet_name='slbdev', engine='openpyxl')
    sign_ub.to_excel(writer, sheet_name='subdev', engine='openpyxl')
    writer.save()

In [49]:
def dfva_main(specsheet, traced, modelfile='../../data/icdf834-mhmc.json', t_max=48, resolution=1):
    """Main dFBA function. Computes successive FBA solutions and plots the concentration, uptake rates, and tracked reaction fluxes.
       
       Arguments:
       specsheet -- path to tsv sheet containing logistic parameters
       traced -- list of reaction ids to be traced over the timecourse
       
       Keyword arguments:
       modelfile -- location of metabolic model
       t_max -- end timepoint in hours (default 48)
       resolution -- number of hours per dFBA sample (default 1)
       """
    print(f"""dFBA log: Begin dFBA analysis with sheet
          {specsheet.split('/')[-1]}, endpoint
          {t_max} hours, and resolution {resolution}.""")
    
    # Load metabolic model and logistic fit specs #
    print('dFBA log: loading model and specsheet...')
    model = cb.io.load_json_model(modelfile)
    model.reactions.ID_90.lower_bound = -1000
    init_cnc = dict()
    for rxn in model.reactions:
        if rxn.id.startswith('Ex_') and rxn.id.endswith('L') or rxn.id in ['Ex_gly', 'Ex_his']:
            init_cnc[rxn.id] = rxn.upper_bound
            rxn.upper_bound *= 0.03
    #model.reactions.ID_53.upper_bound = 0
    #model.reactions.ID_326.upper_bound = 0
    model.reactions.ID_393.upper_bound = 0
    model.reactions.ID_393.lower_bound = 0
    params = pd.read_excel(specsheet, sheet_name='cf', header=0, index_col=0)
    par_lb = pd.read_excel(specsheet, sheet_name='lb', header=0, index_col=0)
    par_ub = pd.read_excel(specsheet, sheet_name='ub', header=0, index_col=0)
    
    # Initialize results dataframes
    timecourse = list(range(0, t_max + resolution, resolution))
    upta_lb = pd.DataFrame(np.zeros(shape=(len(timecourse), par_lb.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_lb.index])
    uptakes = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    upta_ub = pd.DataFrame(np.zeros(shape=(len(timecourse), par_ub.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_ub.index])
    sign_lb = pd.DataFrame(np.zeros(shape=(len(timecourse), par_lb.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_lb.index])
    sign_ub = pd.DataFrame(np.zeros(shape=(len(timecourse), par_ub.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_ub.index])
    signals = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    rxnflux = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced]) # 
    rxnf_lb = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced])
    rxnf_ub = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced])
    
    print('dFBA log: simulation output begin.')
    mets = params.index.to_numpy()
    
    fulldata = [params.to_numpy(), par_lb.to_numpy(), par_ub.to_numpy()]
    traced2 = traced
    if "Ex_biomass" not in traced2:
        traced2.append("Ex_biomass")
    for i, t in enumerate(timecourse):
        for j, met in enumerate(mets):
            data = [[a[j, b] for a in fulldata] for b in range(4)]
            logi, ddxlogi, ulb, uub, slb, sub = update_uptake_bounds(model, t, mets[j], *data)
            uptakes.at[t, model.metabolites.get_by_id(met + '_c').name] = ddxlogi
            upta_lb.at[t, model.metabolites.get_by_id(met + '_c').name] = ulb
            upta_ub.at[t, model.metabolites.get_by_id(met + '_c').name] = uub
            signals.at[t, model.metabolites.get_by_id(met + '_c').name] = logi
            sign_lb.at[t, model.metabolites.get_by_id(met + '_c').name] = slb
            sign_ub.at[t, model.metabolites.get_by_id(met + '_c').name] = sub
        #sol = cb.flux_analysis.pfba(model)
        #sol = cb.flux_analysis.geometric_fba(model)
        #sol = cb.flux_analysis.flux_variability_analysis(model)
        sol_b = model.optimize()
        sol_v = fva(model, traced2, fraction_of_optimum=1., loopless=True)
        if i % 10 == 0:  
            print(f'dFBA log: Time = {t}  (cycle {i}) \tFBA solution: {sol_b.fluxes["Ex_biomass"]}')
        if sol_b.fluxes['Ex_biomass'] < 0.0001:
            print(f'dFBA log: infeasible solution on cycle {i}.')
        for rid in traced:
            rxnflux.at[t, model.reactions.get_by_id(rid).name] = abs(sol_b.fluxes[rid])
            rxnf_ub.at[t, model.reactions.get_by_id(rid).name] = abs(sol_v.at[rid,'maximum'])
            rxnf_lb.at[t, model.reactions.get_by_id(rid).name] = abs(sol_v.at[rid,'minimum'])
        #update_dependent_uptakes(model, sol, resolution)
        if i in [0, 6, 12, 24, 36, 48]:
            # print(model.summary(solution=sol))
            print("time = " + str(i))
#             print(model.summary(solution=sol))
#             print(model.metabolites.get_by_id('gluL_c').summary(solution=sol))
#             print(model.metabolites.get_by_id('serL_c').summary(solution=sol))
#             print(model.metabolites.get_by_id('hpyr_c').summary(solution=sol))
        #         for rxn in model.reactions:
#             if (rxn.id.startswith('Ex_') and rxn.id.endswith('L') or rxn.id in ['Ex_gly', 'Ex_his']):
#                 cnc = init_cnc[rxn.id]
#                 if cnc > 0:
#                     deplete = sol.fluxes[rxn.id] / cnc
#                     rxn.upper_bound *= (1 - deplete)
#                     cnc *= (1 - deplete)
        
            
    print(f'Complete after {i} cycles ({t} hours). Final flux: {sol_b.fluxes["Ex_biomass"]}')
    areaplot(uptakes, upta_lb, upta_ub, ylabel='flux (mM/h)')
    plt.show()
    niceplot(uptakes, ylabel='flux (mM/h)')
    plt.show()
#     plt.savefig('../../data/uptakes.png')
    niceplot(signals, ylabel='concentration (mM)')
    plt.show()
#     plt.savefig('../../data/concentrations.png')
    niceplot(rxnflux)
    plt.show()
#     plt.savefig('../../data/fluxes.png')
    rxnflux.to_csv('../../data/sumfluxes.csv', sep='\t')
#     rxnflux.to_excel('../../data/some_flux.xlsx', engine='xlsxwriter')
    writer = pd.ExcelWriter('../../data/fluxbounds.xlsx', engine='openpyxl')
    uptakes.to_excel(writer, sheet_name='data', engine='openpyxl')
    upta_lb.to_excel(writer, sheet_name='lbdev', engine='openpyxl')
    upta_ub.to_excel(writer, sheet_name='ubdev', engine='openpyxl')
    signals.to_excel(writer, sheet_name='signal', engine='openpyxl')
    sign_lb.to_excel(writer, sheet_name='slbdev', engine='openpyxl')
    sign_ub.to_excel(writer, sheet_name='subdev', engine='openpyxl')
    rxnflux.to_excel(writer, sheet_name='fluxes', engine='openpyxl')
    rxnf_ub.to_excel(writer, sheet_name='fluxub', engine='openpyxl')
    rxnf_lb.to_excel(writer, sheet_name='fluxlb', engine='openpyxl')
    writer.save()

## Run using glucose data

In [51]:
tracked = [
    "ID_391",
    "ID_53",
    "ID_280",
    "ID_326",
    "ID_660",
    "ID_383",
    "ID_336",
    "ID_369",
    "ID_396",
    "RXN-19534",
    "ICCoA-DHG-EB",
    "ID_178",
    "CPLX-8556",
    "ID_603",
    "ID_314",
    "BUK",
    "RNF-Complex",
    "Ex_biomass",
]
#dfba_main("../../data/coeffs.xlsx",
dfva_main("../../data/coeffs.xlsx", 
          tracked,
          t_max=48,
          resolution=1,
         )

dFBA log: Begin dFBA analysis with sheet
          coeffs.xlsx, endpoint
          48 hours, and resolution 1.
dFBA log: loading model and specsheet...
dFBA log: simulation output begin.
dFBA log: Time = 0  (cycle 0) 	FBA solution: 0.030082855898800536
time = 0


KeyboardInterrupt: 

In [ ]:
def pyr_prop(specsheet, traced, modelfile='../../data/icdf834-mhmc.json', t_max=48, resolution=1):
    """Like dfba_main, but writes fluxes for pyruvate reactions."""
    model = cb.io.load_json_model(modelfile)
    model.reactions.ID_90.lower_bound = -1000
    init_cnc = dict()
    for rxn in model.reactions:
        if rxn.id.startswith('Ex_') and rxn.id.endswith('L') or rxn.id in ['Ex_gly', 'Ex_his']:
            init_cnc[rxn.id] = rxn.upper_bound
            rxn.upper_bound *= 0.03
    model.reactions.ID_393.upper_bound = 0
    model.reactions.ID_393.lower_bound = 0
    params = pd.read_csv(specsheet, header=0, index_col=0)
    
    # Initialize results dataframes
    timecourse = list(range(0, t_max + resolution, resolution))
    uptakes = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    signals = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    rxnflux = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced]) # 
    rxnflux2 = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced])
    
    pyr_rxns = []
    pyr_data = []
    
    print('dFBA log: simulation output begin.')
    for i, t in enumerate(timecourse):
        for met, data in params.iterrows():
            logi, ddxlogi = update_uptake_bounds(model, t, met, *data)
            uptakes.at[t, model.metabolites.get_by_id(met + '_c').name] = ddxlogi
            signals.at[t, model.metabolites.get_by_id(met + '_c').name] = logi
        sol = model.optimize()
        if i % 10 == 0:  
            print(f'dFBA log: Time = {t}  (cycle {i}) \tFBA solution: {sol.fluxes["Ex_biomass"]}')
        if sol.fluxes['Ex_biomass'] < 0.0001:
            print(f'dFBA log: infeasible solution on cycle {i}.')
            for rid in traced:
                rxnflux.at[t, model.reactions.get_by_id(rid).name] = 0
        else:
            for rid in traced:
                rxnflux.at[t, model.reactions.get_by_id(rid).name] = abs(sol.fluxes[rid])
        if i in [0, 10, 12, 20, 24, 30, 36, 40, 41, 42, 43, 44, 45, 46, 47, 48]:
            print("time = " + str(i))
            print(model.metabolites.get_by_id('pyr_c').summary(solution=sol))
#             print(model.summary())

        pyr = model.metabolites.pyr_c
        localdata = {}
        for rxn in model.reactions:
            if (pyr in rxn.reactants and sol.fluxes[rxn.id] > 1E-5) or (pyr in rxn.products and sol.fluxes[rxn.id] < -1E-5):
                localdata[rxn.id] = rxn.metabolites[pyr] * sol.fluxes[rxn.id]
        pyr_data.append(localdata)
        for k in localdata:
            pyr_rxns.extend([k for k in localdata])
        
    rids = list(set(pyr_rxns))
    with open('../../data/pyrflux.txt', 'w') as wf:
        wf.write('time\t' + '\t'.join(rids) + '\n')
        for i, t in enumerate(timecourse):
            wf.write(str(t))
            for rxn in rids:
                wf.write('\t' + str(pyr_data[i].get(rxn, 0)))
            wf.write('\n')
        

In [9]:
def ac_prop(specsheet, traced, modelfile="../../data/icdf834-mhmc.json", t_max=48, resolution=1):
    """Like dfba_main, but writes fluxes for acetate reactions.
    Plot with plotFluxArea.m
    """
    model = cb.io.load_json_model(modelfile)
    model.reactions.ID_90.lower_bound = -1000
    init_cnc = dict()
    for rxn in model.reactions:
        if rxn.id.startswith('Ex_') and rxn.id.endswith('L') or rxn.id in ['Ex_gly', 'Ex_his']:
            init_cnc[rxn.id] = rxn.upper_bound
            rxn.upper_bound *= 0.03
    model.reactions.ID_393.upper_bound = 0
    model.reactions.ID_393.lower_bound = 0
    params = pd.read_csv(specsheet, header=0, index_col=0)
    
    # Initialize results dataframes
    timecourse = list(range(0, t_max + resolution, resolution))
    uptakes = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    signals = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    rxnflux = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced]) # 
    rxnflux2 = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced])
    
    ac_rxns_in = []
    ac_data_in = []
    ac_rxns_ot = []
    ac_data_ot = []
    
    print('dFBA log: simulation output begin.')
    for i, t in enumerate(timecourse):
        for met, data in params.iterrows():
            logi, ddxlogi = update_uptake_bounds(model, t, met, *data)
            uptakes.at[t, model.metabolites.get_by_id(met + '_c').name] = ddxlogi
            signals.at[t, model.metabolites.get_by_id(met + '_c').name] = logi
        sol = model.optimize()
        if i % 10 == 0:  
            print(f'dFBA log: Time = {t}  (cycle {i}) \tFBA solution: {sol.fluxes["Ex_biomass"]}')
        if sol.fluxes['Ex_biomass'] < 0.0001:
            print(f'dFBA log: infeasible solution on cycle {i}.')
            for rid in traced:
                rxnflux.at[t, model.reactions.get_by_id(rid).name] = 0
        else:
            for rid in traced:
                rxnflux.at[t, model.reactions.get_by_id(rid).name] = abs(sol.fluxes[rid])
        if i in [0, 12, 20, 24, 36, 48]:
            print("time = " + str(i))
            print(model.metabolites.get_by_id('alaL_c').summary(solution=sol))

        ac = model.metabolites.alaL_c
        localdata_in = {}
        localdata_ot = {}
        for rxn in model.reactions:
            if (ac in rxn.reactants and sol.fluxes[rxn.id] > 1E-5) or (ac in rxn.products and sol.fluxes[rxn.id] < -1E-5):
                localdata_ot[rxn.id] = rxn.metabolites[ac] * sol.fluxes[rxn.id]
            if (ac in rxn.reactants and sol.fluxes[rxn.id] < -1E-5) or (ac in rxn.products and sol.fluxes[rxn.id] > 1E-5):
                localdata_in[rxn.id] = rxn.metabolites[ac] * sol.fluxes[rxn.id]
        ac_data_in.append(localdata_in)
        ac_data_ot.append(localdata_ot)
        for k in localdata_in:
            ac_rxns_in.extend([k for k in localdata_in])
        for k in localdata_ot:
            ac_rxns_ot.extend([k for k in localdata_ot])
        
    rids_in = list(set(ac_rxns_in))
    rids_ot = list(set(ac_rxns_ot))
    with open('../../data/alaflux_in.txt', 'w') as wfi, open('../../data/alaflux_ot.txt', 'w') as wfo:
        wfi.write('time\t' + '\t'.join(rids_in) + '\n')
        wfo.write('time\t' + '\t'.join(rids_ot) + '\n')
        for i, t in enumerate(timecourse):
            wfi.write(str(t))
            wfo.write(str(t))
            for rxn in rids_in:
                wfi.write('\t' + str(ac_data_in[i].get(rxn, 0)))
            for rxn in rids_ot:
                wfo.write('\t' + str(ac_data_ot[i].get(rxn, 0)))
            wfi.write('\n')
            wfo.write('\n')
        

In [10]:
ac_prop("../../data/glucose_coefficients.csv", 
          ['ID_336', 'ID_575', 'ID_660', 'ID_383', 'ID_280', 'ID_685', 'RXN-19534', 'ID_314', 'ID_603', 'ID_369', 'ID_407', 'ID_326', 'ID_90', 'ID_36', 'ID_53', 'RNF-Complex', 'ATPsynth4_1', 
           'ID_28', 'ID_252', 'ID_126', 'ID_582', 'CPLX-8556', 'Ex_biomass'],
          t_max=48,
          resolution=1,
         )

dFBA log: simulation output begin.
dFBA log: Time = 0  (cycle 0) 	FBA solution: 0.010014592488058704
time = 0
alaL_c
Formula: C3H7NO2

Producing Reactions
-------------------
Percent      Flux Reaction                           Definition
 99.67%   0.03847   ID_336 2oglut_c + alaL_c <=> gluL_c + pyr_c
  0.33% 0.0001277   ID_396   pyr_c + serL_c <=> alaL_c + hpyr_c

Consuming Reactions
-------------------
Percent       Flux   Reaction                                                                      Definition
  0.42% -0.0001628    Bio_SPs 0.00033 acoa_c + 0.1161 alaL_c + 0.0288 argL_c + 0.5959 aspL_c + 0.0008 atp_...
  3.89%  -0.001502   Bio_prot 0.375 alaL_c + 0.225 argL_c + 0.441 asnL_c + 0.399 aspL_c + 39.94 atp_c + 0....
  2.41% -0.0009283     ID_506                         alaL_c + atp_c + udpamr_c --> adp_c + pi_c + udpamala_c
 93.28%     -0.036 Tsp_alaL_c                                                               alaL_e <-- alaL_c


/opt/miniconda3/envs/cobrapy/lib/python3.8/site-packages/cobra/util/solver.py:508: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


dFBA log: infeasible solution on cycle 8.
dFBA log: infeasible solution on cycle 9.
dFBA log: Time = 10  (cycle 10) 	FBA solution: 0.0
dFBA log: infeasible solution on cycle 10.
dFBA log: infeasible solution on cycle 11.
dFBA log: infeasible solution on cycle 12.
time = 12
alaL_c
Formula: C3H7NO2

Producing Reactions
-------------------
Percent  Flux Reaction                           Definition
100.00% 0.303   ID_336 2oglut_c + alaL_c <=> gluL_c + pyr_c

Consuming Reactions
-------------------
Percent   Flux   Reaction        Definition
100.00% -0.303 Tsp_alaL_c alaL_e <-- alaL_c
dFBA log: infeasible solution on cycle 13.
dFBA log: infeasible solution on cycle 14.

/opt/miniconda3/envs/cobrapy/lib/python3.8/site-packages/cobra/util/solver.py:508: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)



dFBA log: infeasible solution on cycle 15.
dFBA log: infeasible solution on cycle 16.
dFBA log: infeasible solution on cycle 17.
dFBA log: Time = 20  (cycle 20) 	FBA solution: 0.04985964927735962
time = 20
alaL_c
Formula: C3H7NO2

Producing Reactions
-------------------
Percent   Flux Reaction                           Definition
100.00% 0.1248   ID_336 2oglut_c + alaL_c <=> gluL_c + pyr_c

Consuming Reactions
-------------------
Percent       Flux   Reaction                                                                      Definition
  0.65% -0.0008104    Bio_SPs 0.00033 acoa_c + 0.1161 alaL_c + 0.0288 argL_c + 0.5959 aspL_c + 0.0008 atp_...
  5.99%  -0.007479   Bio_prot 0.375 alaL_c + 0.225 argL_c + 0.441 asnL_c + 0.399 aspL_c + 39.94 atp_c + 0....
  3.70%  -0.004622     ID_506                         alaL_c + atp_c + udpamr_c --> adp_c + pi_c + udpamala_c
 89.66%    -0.1119 Tsp_alaL_c                                                               alaL_e <-- alaL_c
time = 24
alaL_